### Installs & Imports

In [ ]:
!pip install pandas

In [ ]:
import pandas
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Workspace
from azure.identity import DefaultAzureCredential

### Enter your specific AML Details in the config.json file before continuing.
### This step gathers the Default Datastore for your Workspace in order to Upload the Finished File

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
datastore = ws.get_default_datastore()
print(datastore)

### Enter the Datastore URI for your medquad.csv file

In [ ]:
csv_path = ""

In [ ]:
csv_path = "azureml://subscriptions/5c4262a8-9f76-4cda-8365-9c5923f7eba8/resourcegroups/andrewthomas-rg/workspaces/phi3workshop/datastores/workspaceblobstore/paths/UI/2024-11-01_133848_UTC/medquad.csv"

In [ ]:
df = pandas.read_csv(csv_path)

In [ ]:
df.drop(['source','focus_area'], axis=1, inplace=True)

df.loc[1].to_json("row{}.json".format(1))

print(df[:5])

file_to_write = ""
for index in df.index:
    try:
        df.loc[index].to_json("row{}.json".format(index))
        with open("row{}.json".format(index)) as file_handle:
            file_content = file_handle.read()
            file_content = '{"messages": [{"role": "user", "content": ' + file_content[file_content.index(':')+1 : file_content.index(',')] + '}, {"role": "assistant", "content": ' + file_content[file_content.index(':',file_content.index(':'))+1:file_content.rindex('.')+1] + '"}]}'
            file_to_write += file_content + "\n"
            print(index , end='\r')
        if(index == 100):
            break
    except:
        pass

In [ ]:
with open("medquad_short.jsonl","w") as file_handle:
    file_handle.write(file_to_write)

In [ ]:
datastore.upload_files("medquad.jsonl", overwrite=True)
    # target_path='your targetfolder',